In [19]:
import requests
from bs4 import BeautifulSoup
from openai import OpenAI
# import openai
from IPython.display import display, Markdown
from dotenv import load_dotenv
import os

In [20]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')
openai = OpenAI()

In [21]:
# A class to represent a Webpage
# If you're not familiar with Classes, check out the "Intermediate Python" notebook

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [23]:
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [24]:
def user_prompt_for(website):
    user_prompt = f"\nYou are looking at a website titled {website.title}"
    user_prompt += "The contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\
and also provide what are th e curriculam of this certificates. \n\n"
    user_prompt += website.text
    return user_prompt

In [25]:
# print(user_prompt_for(web))

In [26]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [27]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [29]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [30]:
display_summary("https://www.credly.com/org/amazon-web-services/badge/aws-certified-solutions-architect-associate")

# Summary of AWS Certified Solutions Architect – Associate - Credly

The website provides information about the **AWS Certified Solutions Architect – Associate** certification issued by Amazon Web Services (AWS) Training and Certification. This certification is targeted at individuals with an intermediate level of expertise in cloud architecture and demonstrates a comprehensive understanding of AWS services and technologies.

## Key Points
- **Certification Level**: Intermediate
- **Issued By**: Amazon Web Services Training and Certification
- **Skills Acquired**:
  - Amazon Web Services (AWS)
  - Cloud Architecture
  - AWS Cloud Services
  - Cloud Infrastructure
  - Cloud Data Management
  - AWS Certification Strategies

## Earning Criteria
To earn this certification, candidates must successfully pass the AWS Certified Solutions Architect – Associate exam.

This certification verifies the ability to build secure and robust solutions leveraging architectural design principles that fulfill customer requirements. Badge holders are equipped to strategically design well-architected distributed systems that are scalable, resilient, efficient, and fault-tolerant. 

No news or announcements were mentioned on the website.

## Ask question about the website

In [32]:
def ask_question_about(url, user_question):
    website = Website(url)

    messages_2 = [
        {"role": "system", "content": "You are an assistant that answers detailed questions about the content of a website. Use the full page text to help answer questions clearly and accurately."},
        {"role": "user", "content": f"Website content:\n{website.text[:8000]}"},
        {"role": "user", "content": f"My question is: {user_question}"}
    ]

    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages_2
    )

    display(Markdown(response.choices[0].message.content))


In [34]:
ask_question_about(
    "https://aws.amazon.com/certification/certified-solutions-architect-associate/",
    "List the curriculum or key topic areas for this certification"
)


The curriculum or key topic areas for the AWS Certified Solutions Architect - Associate certification include the following domains:

1. **Design Resilient Architectures**: This focuses on designing architectures that can recover from failures and ensure high availability.

2. **Design High-Performing Architectures**: This involves optimizing architectures for performance, enabling quick responses and efficient resource usage.

3. **Design Secure Applications and Architectures**: This includes implementing security best practices, safeguarding data, and designing secure solutions on AWS.

4. **Design Cost-Optimized Architectures**: This domain focuses on designing solutions that minimize costs while maximizing performance and efficiency.

Each of these domains aligns with specific AWS services and best practices, and they are critical for preparing for the certification exam.